In [14]:
import numpy as np 
from implementations import *
from helpers import *
from helper_functions import *
from optimization import *
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
# load the training dataset
print("loading the training dataset")
DATA_TRAIN_PATH = 'train.csv'
y_tr, tx_tr, ids_tr = load_csv_data(DATA_TRAIN_PATH)
print("training dataset loaded")

loading the training dataset
training dataset loaded


In [6]:
# load the test dataset
print("loading the test dataset")
DATA_TEST_PATH = 'test.csv'
y_te, tx_te, ids_te = load_csv_data(DATA_TEST_PATH)
print("test dataset loaded")

loading the test dataset
test dataset loaded


In [7]:
# printing the shapes of the arrays
print("the shape of the training y is ",y_tr.shape," and the training tx is ", tx_tr.shape)
print("the shape of the test y is ",y_te.shape," and the test tx is ",tx_te.shape)

the shape of the training y is  (250000,)  and the training tx is  (250000, 30)
the shape of the test y is  (568238,)  and the test tx is  (568238, 30)


In [5]:
# testing the mean square gd
initial_w = np.zeros(tx_tr.shape[1])
max_iters = 500
gamma = 0.00008

#w, loss = mean_squared_error_gd(y_tr, tx_tr, initial_w, max_iters, gamma)

#print("end of the mean_squared_error_gd with w=",w," and loss=", loss)

In [8]:
# test least square
txOpti = dataClean_without_splitting(tx_tr)
w, loss = least_squares(y_tr, txOpti)
print("end of the least_squares with w=",w," and loss=", loss)

end of the least_squares with w= [-3.14664000e-01 -6.17391433e-02 -2.63661630e-01  4.00443166e-02
 -3.23831409e-03  2.36819497e+01 -1.06665608e-01 -1.31956774e+01
  1.10025577e-01 -4.10429034e-02  1.47792868e-02 -7.39266340e-02
  1.29692142e-01 -1.91168474e+00  1.51246931e-01 -1.97869684e-03
 -5.69064483e-04  5.85794408e-02 -5.37734703e-04  3.94036228e-03
  4.26378298e-02  2.24787819e-03  2.81790068e-03 -1.56586861e-01
  1.87663734e-02  9.31168600e-03  1.26628842e-02  1.49544782e-01
 -2.93365581e+00 -5.58783256e+00  5.36587380e-02]  and loss= 0.3393721979818556


In [7]:
# test mean squared error sgd

initial_w = np.zeros(tx_tr.shape[1])
max_iters = 100
gamma = 0.00001

#w, loss = mean_squared_error_sgd(y_tr, tx_tr, initial_w, max_iters, gamma)

#print("end of the mean_squared_error_sgd with w=",w," and loss=", loss)

In [9]:
# test ridge regression 

txOpti = dataClean_without_splitting(tx_tr)
print("shape of txOpti ", txOpti.shape)

lambda_ = best_lambda(y_tr, txOpti, -6, 0, 30)
degree = 2
print("best lambda is ",lambda_)

txOpti_poly = build_poly(txOpti, degree)

w, loss = ridge_regression(y_tr, txOpti_poly, lambda_)

label = predict(w, txOpti_poly)
acc = calculate_accuracy(y_tr, label)

print("end of the logistic_regression with w=",w," and loss=", loss)
print("the accuracy on the train set is ", acc)

#y_pred = predict(w, tx_te)
#y_pred[y_pred==0] = -1

#OUTPUT_PATH = 'sample-submission'
#create_csv_submission(ids_te, y_pred, OUTPUT_PATH)

shape of txOpti  (250000, 31)
Current lambda=1e-06, the loss=0.33948075708431097
Current lambda=1.6102620275609392e-06, the loss=0.33957303863352495
Current lambda=2.592943797404667e-06, the loss=0.3397317059940319
Current lambda=4.1753189365604e-06, the loss=0.3399849888270534
Current lambda=6.723357536499335e-06, the loss=0.3403502544581446
Current lambda=1.082636733874054e-05, the loss=0.3408127985026071
Current lambda=1.7433288221999873e-05, the loss=0.3413191893017609
Current lambda=2.8072162039411757e-05, the loss=0.3418003259438394
Current lambda=4.520353656360241e-05, the loss=0.34220482264575247
Current lambda=7.278953843983146e-05, the loss=0.3425137911381104
Current lambda=0.00011721022975334806, the loss=0.34273389111920344
Current lambda=0.00018873918221350977, the loss=0.34288337153176474
Current lambda=0.0003039195382313198, the loss=0.3429818717475476
Current lambda=0.0004893900918477494, the loss=0.3430458955636129
Current lambda=0.0007880462815669912, the loss=0.34308

In [31]:
# test logistic regression

txOpti, yOpti = dataClean(tx_tr, y_tr)
print("shape of txOpti ", txOpti[0].shape)
print("shape of yOpti ", txOpti[0].shape)
#initial_w = np.zeros(txOpti.shape[1])
# initial_w = w
max_iters = 100
gamma = 0.5
ws = []
losses = []
y_pred = []
accArray = []

for i in range(4): 
    initial_w = np.zeros(txOpti[i].shape[1])
    w, loss = logistic_regression(yOpti[i], txOpti[i], initial_w, max_iters, gamma)
    label = predict(w, txOpti[i])
    acc = calculate_accuracy(yOpti[i], label)
    
    ws.append(w)
    losses.append(loss)
    y_pred.append(label)
    accArray.append(acc)

tot_acc = accArray[0] + accArray[1] + accArray[2] + accArray[3]
weights = np.array(list(ws[0])+list(ws[1])+list(ws[2])+list(ws[3]))
loss = np.mean(losses)
    
#teOpti = dataClean(tx_te)
#print("shape of teOpti ", teOpti.shape)

#y_pred = predict(w, teOpti)

#OUTPUT_PATH = 'sample-submission LR 2'
#create_csv_submission(ids_te, y_pred, OUTPUT_PATH)

print("end of the logistic_regression with w=",weights," and loss=", loss)
print("the accurcy on the train set is ", tot_acc)

shape of txOpti  (99913, 20)
shape of yOpti  (99913, 20)
Preiteration, the loss=1.1116528187695083, the grad=210.0425291113847


/home/ml-project-1-ahl/helper_functions.py:61: RuntimeWarning: divide by zero encountered in log
  return -np.sum(y * np.log(sigmoid(v)) + (1 - y) * np.log(1-sigmoid(v)))/N
/usr/local/lib/python3.8/dist-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


Current iteration=0, the loss=nan, the grad=210.0425291113847
Current iteration=1, the loss=nan, the grad=103.84048931520206
Current iteration=2, the loss=nan, the grad=103.84048931520206
Current iteration=3, the loss=nan, the grad=103.84048931520206
Current iteration=4, the loss=nan, the grad=103.84048931520206
Current iteration=5, the loss=nan, the grad=103.84048931520206
Current iteration=6, the loss=nan, the grad=103.84048931520206
Current iteration=7, the loss=nan, the grad=103.84048931520206
Current iteration=8, the loss=nan, the grad=103.84048931520206
Current iteration=9, the loss=nan, the grad=103.84048931520206
Current iteration=10, the loss=nan, the grad=103.84048931520206
Current iteration=11, the loss=nan, the grad=103.84048931520206
Current iteration=12, the loss=nan, the grad=103.84048931520206
Current iteration=13, the loss=nan, the grad=103.84048931520206
Current iteration=14, the loss=nan, the grad=103.84048931520206
Current iteration=15, the loss=nan, the grad=103.84

KeyboardInterrupt: 

In [10]:
# test logistic regression without splitting 
txOpti = dataClean_without_splitting(tx_tr)
print("shape of txOpti ", txOpti.shape)
print("shape of y ", y_tr.shape)
initial_w = np.zeros(txOpti.shape[1])
max_iters = 100
gamma = 0.5

w, loss = logistic_regression(y_tr, txOpti, initial_w, max_iters, gamma)

label = predict_logistic(w, txOpti)
acc = calculate_accuracy(y_tr, label)

print("end of the logistic_regression with w=",w," and loss=", loss)
print("the accuracy on the train set is ", acc)

shape of txOpti  (250000, 31)
shape of y  (250000,)
Preiteration, the loss=1.0624754799064415, the grad=1.1261387558041078
Current iteration=0, the loss=0.679184067185061, the grad=1.1261387558041078
Current iteration=1, the loss=0.4702392454195871, the grad=0.8538167005002987
Current iteration=2, the loss=0.24733569478501988, the grad=0.7566691907262045
Current iteration=3, the loss=0.03965120507362577, the grad=0.7032345486116555
Current iteration=4, the loss=-0.15856309305254304, the grad=0.6682047651428545
Current iteration=5, the loss=-0.35012637130185936, the grad=0.6444012668452963
Current iteration=6, the loss=-0.5369500971543492, the grad=0.627674520555563
Current iteration=7, the loss=-0.7202943361917574, the grad=0.6155608377377577
Current iteration=8, the loss=-0.9010072801331301, the grad=0.6065432404283911
Current iteration=9, the loss=-1.0796755361909431, the grad=0.5996599381310498
Current iteration=10, the loss=-1.2567164527783987, the grad=0.5942854181664731
Current i

In [23]:
#clean column , remove column that has only -999 elements. 
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})

median = np.median(tx_tr, axis=0) 
mean = np.mean(tx_tr, axis=0) 

print(median)
print(mean)

indx = np.where(median == -999)
print(indx)

counts = []
count = 0

for i in indx[0]: 
    count = np.count_nonzero( tx_tr[:,i] == -999)
    print("index :", i)
    print(count)
    counts.append(count)

print("counts of each column ", counts)

[105.012 46.524 73.752 38.468 -999.000 -999.000 -999.000 2.492 12.316
 120.665 1.280 -0.356 -999.000 31.804 -0.023 -0.033 40.516 -0.045 0.086
 34.802 -0.024 179.739 1.000 38.960 -1.872 -2.093 -999.000 -999.000
 -999.000 40.513]
[-49.023 49.240 81.182 57.896 -708.421 -601.237 -709.357 2.373 18.917
 158.432 1.438 -0.128 -708.985 38.707 -0.011 -0.008 46.660 -0.020 0.044
 41.717 -0.010 209.797 0.979 -348.330 -399.254 -399.260 -692.381 -709.122
 -709.119 73.065]
(array([ 4,  5,  6, 12, 26, 27, 28]),)
index : 4
177457
index : 5
177457
index : 6
177457
index : 12
177457
index : 26
177457
index : 27
177457
index : 28
177457
counts of each column  [177457, 177457, 177457, 177457, 177457, 177457, 177457]


In [27]:
median = np.median(tx_tr, axis=0) 
indx = np.where(median == -999)
tx1 = np.delete(tx_tr, indx[0], 1)
print(indx[0].shape)
print(tx_tr.shape)
print(tx1.shape)

(7,)
(250000, 30)
(250000, 23)


In [16]:
# test regularized logistic regression
txOpti = dataClean_without_splitting(tx_tr)
print("shape of txOpti ", txOpti.shape)
print("shape of y ", y_tr.shape)
initial_w = np.zeros(txOpti.shape[1])
max_iters = 100
gamma = 0.5
lambda_ = 0.5

w, loss = reg_logistic_regression(y_tr, txOpti, lambda_, initial_w, max_iters, gamma)

label = predict_logistic(w, txOpti)
acc = calculate_accuracy(y_tr, label)

print("end of the logistic_regression with w=",w," and loss=", loss)
print("the accuracy on the train set is ", acc)

teOpti = dataClean_without_splitting(tx_te)
print("shape of teOpti ", teOpti.shape)

y_pred = predict_logistic(w, teOpti)

OUTPUT_PATH = 'sample-submission LR regression r=1.5, gamma,lambda=0.5'
create_csv_submission(ids_te, y_pred, OUTPUT_PATH)

shape of txOpti  (250000, 24)
shape of y  (250000,)
Current iteration=0, the loss=0.7070366274465562, the grad=1.0991914599924302
Current iteration=1, the loss=0.6223439392176738, the grad=0.357515024179797
Current iteration=2, the loss=0.5832462612934024, the grad=0.13691403689386653
Current iteration=3, the loss=0.5679408656189062, the grad=0.055114316590955496
Current iteration=4, the loss=0.561741397073609, the grad=0.022421000076055977
Current iteration=5, the loss=0.5592254737046597, the grad=0.009165963245399178
Current iteration=6, the loss=0.5582009730840245, the grad=0.003758517828891186
Current iteration=7, the loss=0.5577828898901592, the grad=0.00154497753493535
Current iteration=8, the loss=0.5576119844514998, the grad=0.0006366126656023305
Current iteration=9, the loss=0.5575420204953335, the grad=0.0002630022845962372
Current iteration=10, the loss=0.5575133423312943, the grad=0.00010897088488415756
Current iteration=11, the loss=0.557501573049589, the grad=4.5299426082

In [5]:
jet = tx_tr[:, 22]

values = []

for x in jet: 
    if x not in values: 
        values.append(x) 

print(values) 

[2.0, 1.0, 0.0, 3.0]
